<a href="https://colab.research.google.com/github/taniokah/DL-Basic-Seminar/blob/master/AI%E3%83%96%E3%83%BC%E3%83%A0%E5%88%86%E6%9E%90.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#AIブーム分析

https://ohke.hateblo.jp/entry/2019/06/01/120000
https://qiita.com/a2kiti/items/a93d762bd89a84316c12

1. まず、Sudachiのword2vecモデルをダウンロードする
2. 対象となる新聞記事を読み込む
3. 新聞記事からAIに近い単語を上位10個ずつ抽出する
4. 単語ベクトルとその頻度のリストを作る
5. 時系列トピックモデルを用いて分析する


In [0]:
!wget https://object-storage.tyo2.conoha.io/v1/nc_2520839e1f9641b08211a5c85243124a/word_vector/nwjc.sudachi_full_abc_w2v.20190314.tar.gz

--2020-01-29 23:58:02--  https://object-storage.tyo2.conoha.io/v1/nc_2520839e1f9641b08211a5c85243124a/word_vector/nwjc.sudachi_full_abc_w2v.20190314.tar.gz
Resolving object-storage.tyo2.conoha.io (object-storage.tyo2.conoha.io)... 157.7.224.17
Connecting to object-storage.tyo2.conoha.io (object-storage.tyo2.conoha.io)|157.7.224.17|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5212195768 (4.9G) [application/x-www-form-urlencoded]
Saving to: ‘nwjc.sudachi_full_abc_w2v.20190314.tar.gz’

wjc.sudachi_full_ab  20%[===>                ]   1.01G  74.7MB/s    eta 54s    

In [2]:
!tar -zxvf ./nwjc.sudachi_full_abc_w2v.20190314.tar.gz

nwjc_sudachi_full_abc_w2v/
nwjc_sudachi_full_abc_w2v/nwjc.sudachi_full_abc_w2v.txt
nwjc_sudachi_full_abc_w2v/LICENSE


In [23]:
!ls -l nwjc_sudachi_full_abc_w2v/

total 12229548
-rw-rw-r-- 1 1008 1008       11420 Mar 20  2019 LICENSE
-rw-rw-r-- 1 1008 1008 12523039651 Mar 20  2019 nwjc.sudachi_full_abc_w2v.txt


In [24]:
!head -n2 nwjc_sudachi_full_abc_w2v/nwjc.sudachi_full_abc_w2v.txt

3644628 300
の -0.08509623 -0.02170456 -0.00595318 0.17987297 -0.02470185 -0.06039717 0.16489118 0.18576832 -0.13870764 -0.17454691 0.08653757 -0.11530939 -0.0357993 0.0308698 -0.11082912 0.04128339 0.00738431 0.00342017 -0.00673456 0.03348965 -0.18876095 0.05184764 0.13508447 0.24196455 -0.07351201 -0.07903086 -0.01973874 -0.24246444 0.03141621 -0.11729659 -0.04126292 -0.20383213 -0.03109288 0.06765312 -0.15020953 -0.11498244 0.03988069 0.20287995 -0.14308368 -0.05135961 -0.15592268 0.05364651 -0.11668575 0.0505892 -0.04575419 0.03296537 -0.03033004 -0.0481239 0.13578212 -0.03111291 -0.15257519 -0.10075521 -0.07440718 -0.12524064 -0.14464405 -0.00155862 0.11825287 0.01658496 0.27135071 -0.07724451 0.01130517 0.03584538 0.01659033 0.19819288 0.01558947 0.07239497 -0.05149494 0.08688559 -0.03712051 -0.07294089 -0.00820937 -0.00907754 -0.03328617 -0.09273465 0.07537545 -0.05137397 0.07578564 -0.03124459 0.05484971 -0.07136369 -0.02457538 0.15481928 0.11072636 -0.00225826 -0.04312619 -0.09

In [0]:
#!pip install gensim

In [0]:
# （今回は使わない）

# モデルのロード
from gensim.models import KeyedVectors
from gensim.test.utils import datapath

nwjc_model = KeyedVectors.load_word2vec_format('/content/nwjc_sudachi_full_abc_w2v/nwjc.sudachi_full_abc_w2v.txt', binary=False)

#nwjc_model = KeyedVectors.load_word2vec_format(
#    datapath('/content/nwjc_sudachi_full_abc_w2v/nwjc.sudachi_full_abc_w2v.txt'),
#    binary=False
#)

# 語数, 次元数
print(len(nwjc_model.vocab), nwjc_model.vector_size)  # 3644628 300

In [0]:
# （今回は使わない）

print(nwjc_model.most_similar('平成', topn=5))
# [('年度', 0.8092531561851501),
#  ('平成元年', 0.7409119009971619),
#  ('同年', 0.6042426824569702),
#  ('基本計画', 0.5917632579803467),
#  ('別紙', 0.590812087059021)]

print(nwjc_model.most_similar(positive=['兄弟', '女'], negative=['男'], topn=5))
# [('姉妹', 0.7687463164329529),
#  ('兄妹', 0.7484474182128906),
#  ('姉弟', 0.715774416923523),
#  ('双子', 0.642593264579773),
#  ('妹', 0.6402267217636108)]

In [3]:
!pip install SudachiPy

     |████████████████████████████████| 81kB 2.5MB/s 
  Created wheel for dartsclone: filename=dartsclone-0.6-cp36-cp36m-linux_x86_64.whl size=413256 sha256=0c6dc0f632957a8dc4d5f55d8816d2e7cbfeb2657d56bb55b0735a9d52ad6d37
  Stored in directory: /root/.cache/pip/wheels/be/cd/70/fe43307bf7398243155108f4f5a258ef336923d65ec4af93cd
Successfully built dartsclone


In [4]:
!pip install https://object-storage.tyo2.conoha.io/v1/nc_2520839e1f9641b08211a5c85243124a/sudachi/SudachiDict_core-20191224.tar.gz

     |████████████████████████████████| 70.7MB 61kB/s 
  Created wheel for SudachiDict-core: filename=SudachiDict_core-20191224-cp36-none-any.whl size=70876855 sha256=13d033b6c96ae617b096144f2c47f1cfa957974baa6e23fd4fd79eeb04e5bfa5
  Stored in directory: /root/.cache/pip/wheels/4c/88/5c/44c0deb66c147b881dab3fe061febbb301bdceab7e9b496479
Successfully built SudachiDict-core


In [27]:
!sudachipy tokenize -h

usage: sudachipy tokenize [-h] [-r file] [-m {A,B,C}] [-o file] [-a] [-d] [-v]
                          [file [file ...]]

Tokenize Text

positional arguments:
  file           text written in utf-8

optional arguments:
  -h, --help     show this help message and exit
  -r file        the setting file in JSON format
  -m {A,B,C}     the mode of splitting
  -o file        the output file
  -a             print all of the fields
  -d             print the debug information
  -v, --version  print sudachipy version


In [14]:
!pip install mojimoji

  Created wheel for mojimoji: filename=mojimoji-0.0.9-cp36-cp36m-linux_x86_64.whl size=117567 sha256=12c8895291a04dbef55d4fddde1c8fa8df4253b1a0d87a393d0fd6ea3521a756
  Stored in directory: /root/.cache/pip/wheels/67/f0/6e/03e5c5017afc8230e19abeacad57189138207fea2f5dd71880
Successfully built mojimoji


In [99]:
# 前もって sudachiData.pickle をアップロードしておくと起動が速い

import numpy as np
import pickle
import re
import csv
import os
from sudachipy import tokenizer
from sudachipy import dictionary
from sklearn.metrics.pairwise import cosine_similarity
import mojimoji

class Sudachi_w2v:
    def __init__(self, path, sudachiDataPath="sudachiData.pickle"):
        self.cache = {}
        f = open(path, 'r')
        self.file = f
        self.reader = csv.reader(f, delimiter=' ')
        #最初に含有単語リストやメモリアドレスリストを作成する（かなり時間かかる）
        #2回目以降はpickle化したものを読み込む
        if os.path.exists(sudachiDataPath):
            with open(sudachiDataPath, 'rb') as f:
                dataset = pickle.load(f)
            self.offset_list = dataset["offset_list"]
            self.emb_size = dataset["emb_size"]
            self.word2index = dataset["word2index"]
            self.ave_vec = dataset["ave_vec"]
        else:
            txt = f.readline()
            #分散表現の次元数
            self.emb_size = int(txt.split()[1])
            #未知語が来た場合平均ベクトルを返す
            self.ave_vec = np.zeros(self.emb_size, np.float)
            #メモリアドレスリスト
            self.offset_list = []
            word_list = []
            count = 0
            maxCount = int(txt.split()[0])
            while True:
                count+=1
                self.offset_list.append(f.tell())
                if count % 100000 == 0:print(count,"/",maxCount)
                line = f.readline()
                if line == '':break
                line_list = line.split()
                word_list.append(line_list[0])
                self.ave_vec += np.array(line_list[-300:]).astype(np.float)
            self.offset_list.pop()
            self.ave_vec = self.ave_vec/count
            self.word2index = {v:k for k,v in enumerate(word_list)}

            dataset = {}
            dataset["offset_list"] = self.offset_list
            dataset["emb_size"] = self.emb_size
            dataset["word2index"] = self.word2index
            dataset["ave_vec"] = self.ave_vec
            with open(sudachiDataPath, 'wb') as f:
                pickle.dump(dataset, f)

        self.num_rows = len(self.offset_list)
        #sudachiの準備
        self.tokenizer_obj = dictionary.Dictionary().create()
        self.mode = tokenizer.Tokenizer.SplitMode.C

    #単語をベクトル化
    def word2vec(self, word):
        try:
            idx = self.word2index[word]
            result = self.read_row(idx)
            vec = np.array(result[-300:])
            return vec
        except:#単語リストにない場合
            print(word, ": out of wordlist")

    #文章を分かち書きした後，それぞれのベクトルをmatでまとめて返す
    def sentence2mat(self, sentence):
        words = sentence.replace("　"," ").replace("\n"," ")
        words = re.sub(r"\s+", " ", words)
        input_seq = [mojimoji.zen_to_han(m.surface()) for m in self.tokenizer_obj.tokenize(words, self.mode)]
        input_seq = [s.lower() for s in input_seq]
        input_seq = [s for s in input_seq if s != ' ']

        mat = np.zeros((len(input_seq), self.emb_size))
        input_sentence = []
        for i, word in enumerate(input_seq):
            try:
                idx = self.word2index[word]
                result = self.read_row(idx)
                input_sentence.append(result[0])
                mat[i] = np.array(result[-300:])
            except:#単語リストにない場合平均ベクトルを返す
                input_sentence.append("<UNK>")
                mat[i] = self.ave_vec
        return input_sentence, mat

    def __del__(self):
        self.file.close()

    def read_row(self, idx):
        self.file.seek(self.offset_list[idx])
        return next(self.reader)
    
    def cos(self, vec1, vec2):
        key = string(vec1) + string(vec2)
        if (key in cache.keys()): 
          return cache[key]
        else 
          similarity = round(float(cosine_similarity([vec1], [vec2])[0][0]), 6)
          cache[key] = similarity
          return similarity

SyntaxError: ignored

In [0]:
path = '/content/nwjc_sudachi_full_abc_w2v/nwjc.sudachi_full_abc_w2v.txt'
sudachi = Sudachi_w2v(path)

In [0]:
vec = sudachi.word2vec("すだち")
print(vec)
#['0.07975651' '0.08931299' '-0.06070593' '0.46959993' '0.19651023' ~

input_sentence, mat = sudachi.sentence2mat("あきらめたらそこで試合終了だよ")
print(input_sentence)
print(mat)
#(['あきらめ', 'たら', 'そこ', 'で', '試合終了', 'だ', 'よ'], array([[ 1.9788130e-02,  1.1190426e-01, -1.6153505e-01, ...,

input_sentence, mat = sudachi.sentence2mat("人工知能（ＡＩ）の進化を受け、データの活用法は企業の競争力を左右する。")
print(input_sentence)
print(mat)

input_sentence, mat = sudachi.sentence2mat("AIってなんだろ？")
print(input_sentence)
print(mat)


In [0]:
# 新聞記事をNK以下にアップロードする

#!mkdir NK
#!mv NKM*.csv NK/

## 新聞記事を読み込む

In [0]:
import csv

with open("/content/NK/NKM_2018.csv", newline="") as f:
  reader = csv.reader(f, delimiter=",", quotechar='"')

  count = 0
  label = []
  for row in reader:
    count += 1
    if count == 1:
      label.append(row[9])
      label.append(row[10])
      continue
    elif count >= 10:
      break
    print(label[0], row[9])
    print(label[1], row[10])
    print('\n')


In [0]:
# SudachiPyで解析してmatを返す

with open("/content/NK/NKM_2018.csv", newline="") as f:
  reader = csv.reader(f, delimiter=",", quotechar='"')

  count = 0
  label = []
  for row in reader:
    count += 1
    if count == 1:
      label.append(row[9])
      label.append(row[10])
      continue
    elif count >= 10:
      break
    input_sentence, mat = sudachi.sentence2mat(row[9])
    print(label[0], input_sentence)
    #print(mat)
    input_sentence, mat = sudachi.sentence2mat(row[10])
    print(label[1], input_sentence)
    #print(mat)
    print('\n')


In [43]:
from sklearn.metrics.pairwise import cosine_similarity

similarity = round(float(cosine_similarity([[1, 0, -1]], [[-1,-1, 0]])[0][0]), 6)
print(similarity)

similarity = round(float(cosine_similarity([[-0.08247425,  0.17477778,  0.08027521,  0.32425445,  0.17314065,  -0.10243454]], 
                                     [[-0.04262996,  0.08501831,  0.01364794,  0.00316689,  0.04561372,  0.30977321]])[0][0]), 6)
print(similarity)

similarity = round(float(cosine_similarity([[-0.02831401,  0.10125325, -0.12704873, 0.35899675,  0.03712214,   0.02951937]], 
                                     [[-0.04262996,  0.08501831,  0.01364794,  0.00316689,  0.04561372,  0.30977321]])[0][0]), 6)
print(similarity)

similarity = round(float(cosine_similarity([[-0.04262996,  0.08501831,  0.01364794,  0.00316689,  0.04561372,  0.30977321]], 
                                     [[-0.04262996,  0.08501831,  0.01364794,  0.00316689,  0.04561372,  0.30977321]])[0][0]), 5)
print(similarity)

-0.5
-0.023404
0.153889
1.0


In [33]:
def cos(word):
  vec1 = sudachi.word2vec(word[0])
  vec2 = sudachi.word2vec(word[1])
  sim = sudachi.cos(vec1, vec2)
  print("cos(" + word[0] + ", " + word[1] + ") \n =>", sim)

cos(["ai", "人工知能"])
cos(["ai", "機械学習"])
cos(["人工知能", "機械学習"])
cos(["人工知能", "自然言語処理"])
cos(["機械学習", "自然言語処理"])
cos(["ai", "自然言語処理"])


cos(ai, 人工知能) 
 => 0.595735
cos(ai, 機械学習) 
 => 0.378244
cos(人工知能, 機械学習) 
 => 0.57597
cos(人工知能, 自然言語処理) 
 => 0.569241
cos(機械学習, 自然言語処理) 
 => 0.641625
cos(ai, 自然言語処理) 
 => 0.327691


In [0]:
# SudachiPyで解析して"AI"に対するcosを返す

import csv

def coscount(filedir, fname, query, thres):
  ai = sudachi.word2vec(query)

  with open(filedir + fname + "_AI.csv", 'w', newline='') as fout:
      writer = csv.writer(fout, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
      writer.writerow(['date', 'mediacode', 'headbody', 'word', 'cosine'])

      total = 0
      with open(filedir + fname + ".csv", newline="") as fin:
        reader = csv.reader(fin, delimiter=",", quotechar='"')

        count = 0
        label = []
        for row in reader:
          count += 1
          if count == 1:
            label.append(row[9])
            label.append(row[10])
            continue
          
          input_sentence, mat = sudachi.sentence2mat(row[9])
          i = 0
          for vec in mat:
            sim = sudachi.cos(ai, vec)
            if (sim > thres):
              #print(count, label[0], input_sentence[i], sim) #, vec)
              #print([row[1], row[3], label[0], input_sentence[i], sim])
              writer.writerow([row[1], row[3], label[0], input_sentence[i], sim])
              total += 1
            i += 1

          input_sentence, mat = sudachi.sentence2mat(row[10])
          i = 0
          for vec in mat:
            sim = sudachi.cos(ai, vec)
            if (sim > thres):
              #print(count, label[1], input_sentence[i], sim) #, vec)
              #print([row[1], row[3], label[1], input_sentence[i], sim])
              writer.writerow([row[1], row[3], label[1], input_sentence[i], sim])
              total += 1
            i += 1

  return total

In [0]:
coscount("/content/NK/", "NKM_2000", "人工知能", 0.5)
coscount("/content/NK/", "NKM_2001", "人工知能", 0.5)
coscount("/content/NK/", "NKM_2002", "人工知能", 0.5)
coscount("/content/NK/", "NKM_2003", "人工知能", 0.5)
coscount("/content/NK/", "NKM_2004", "人工知能", 0.5)
coscount("/content/NK/", "NKM_2005", "人工知能", 0.5)
coscount("/content/NK/", "NKM_2006", "人工知能", 0.5)
coscount("/content/NK/", "NKM_2007", "人工知能", 0.5)
coscount("/content/NK/", "NKM_2008", "人工知能", 0.5)
coscount("/content/NK/", "NKM_2009", "人工知能", 0.5)
coscount("/content/NK/", "NKM_2010", "人工知能", 0.5)
coscount("/content/NK/", "NKM_2011", "人工知能", 0.5)
coscount("/content/NK/", "NKM_2012", "人工知能", 0.5)
coscount("/content/NK/", "NKM_2013", "人工知能", 0.5)
coscount("/content/NK/", "NKM_2014", "人工知能", 0.5)
coscount("/content/NK/", "NKM_2015", "人工知能", 0.5)
coscount("/content/NK/", "NKM_2016", "人工知能", 0.5)
coscount("/content/NK/", "NKM_2017", "人工知能", 0.5)
coscount("/content/NK/", "NKM_2018", "人工知能", 0.5)

coscount("/content/NK/", "NKL_2000", "人工知能", 0.5)
coscount("/content/NK/", "NKL_2001", "人工知能", 0.5)
coscount("/content/NK/", "NKL_2002", "人工知能", 0.5)
coscount("/content/NK/", "NKL_2003", "人工知能", 0.5)
coscount("/content/NK/", "NKL_2004", "人工知能", 0.5)
coscount("/content/NK/", "NKL_2005", "人工知能", 0.5)
coscount("/content/NK/", "NKL_2006", "人工知能", 0.5)
coscount("/content/NK/", "NKL_2007", "人工知能", 0.5)
coscount("/content/NK/", "NKL_2008", "人工知能", 0.5)
coscount("/content/NK/", "NKL_2009", "人工知能", 0.5)
coscount("/content/NK/", "NKL_2010", "人工知能", 0.5)
coscount("/content/NK/", "NKL_2011", "人工知能", 0.5)
coscount("/content/NK/", "NKL_2012", "人工知能", 0.5)
coscount("/content/NK/", "NKL_2013", "人工知能", 0.5)
coscount("/content/NK/", "NKL_2014", "人工知能", 0.5)
coscount("/content/NK/", "NKL_2015", "人工知能", 0.5)
coscount("/content/NK/", "NKL_2016", "人工知能", 0.5)
coscount("/content/NK/", "NKL_2017", "人工知能", 0.5)
coscount("/content/NK/", "NKL_2018", "人工知能", 0.5)


In [0]:
from google.colab import files

files.download("/content/NK/NKM_2000_AI.csv")
files.download("/content/NK/NKM_2001_AI.csv")
files.download("/content/NK/NKM_2002_AI.csv")
files.download("/content/NK/NKM_2003_AI.csv")
files.download("/content/NK/NKM_2004_AI.csv")
files.download("/content/NK/NKM_2005_AI.csv")
files.download("/content/NK/NKM_2006_AI.csv")
files.download("/content/NK/NKM_2007_AI.csv")
files.download("/content/NK/NKM_2008_AI.csv")
files.download("/content/NK/NKM_2009_AI.csv")
files.download("/content/NK/NKM_2010_AI.csv")
files.download("/content/NK/NKM_2011_AI.csv")
files.download("/content/NK/NKM_2012_AI.csv")
files.download("/content/NK/NKM_2013_AI.csv")
files.download("/content/NK/NKM_2014_AI.csv")
files.download("/content/NK/NKM_2015_AI.csv")
files.download("/content/NK/NKM_2016_AI.csv")
files.download("/content/NK/NKM_2017_AI.csv")
files.download("/content/NK/NKM_2018_AI.csv")

files.download("/content/NK/NKL_2000_AI.csv")
files.download("/content/NK/NKL_2001_AI.csv")
files.download("/content/NK/NKL_2002_AI.csv")
files.download("/content/NK/NKL_2003_AI.csv")
files.download("/content/NK/NKL_2004_AI.csv")
files.download("/content/NK/NKL_2005_AI.csv")
files.download("/content/NK/NKL_2006_AI.csv")
files.download("/content/NK/NKL_2007_AI.csv")
files.download("/content/NK/NKL_2008_AI.csv")
files.download("/content/NK/NKL_2009_AI.csv")
files.download("/content/NK/NKL_2010_AI.csv")
files.download("/content/NK/NKL_2011_AI.csv")
files.download("/content/NK/NKL_2012_AI.csv")
files.download("/content/NK/NKL_2013_AI.csv")
files.download("/content/NK/NKL_2014_AI.csv")
files.download("/content/NK/NKL_2015_AI.csv")
files.download("/content/NK/NKL_2016_AI.csv")
files.download("/content/NK/NKL_2017_AI.csv")
files.download("/content/NK/NKL_2018_AI.csv")
